In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

df_firstRound = pd.read_csv('French_Presidential_Election_2017_First_Round.csv')
df_secondRound = pd.read_csv('French_Presidential_Election_2017_Second_Round.csv')

df_baseEtabl = pd.read_csv('base_etablissement_par_tranche_effectif.csv')
df_geoInfo = pd.read_csv('name_geographic_information.csv')
df_salaryInfo = pd.read_csv('net_salary_per_town_categories.csv')
df_population = pd.read_csv('population.csv')

In [2]:
#Clean INSEE code for df_firstRound and other datasets
df_firstRound['Z'] = df_firstRound['INSEE code'].apply(lambda x: 'Z' if str(x)[0] == 'Z' else 'No')
df_firstRound['AB'] = df_firstRound['INSEE code'].apply(lambda x: 'AB' if str(x)[0:2] == '2A' or str(x)[0:2] == '2B' else 'No')
df_firstRound[df_firstRound['AB'] == 'AB']

df_firstRound_Z = df_firstRound[df_firstRound['Z'] == 'Z']
df_firstRound_AB = df_firstRound[df_firstRound['AB'] == 'AB']


#Clean INSEE code for df_SecondRound
df_secondRound['Z'] = df_secondRound['INSEE code'].apply(lambda x: 'Z' if str(x)[0] == 'Z' else 'No')
df_secondRound['AB'] = df_secondRound['INSEE code'].apply(lambda x: 'AB' if str(x)[0:2] == '2A' or str(x)[0:2] == '2B' else 'No')
df_secondRound[df_secondRound['AB'] == 'AB']

df_secondRound_Z = df_secondRound[df_secondRound['Z'] == 'Z']
df_secondRound_AB = df_secondRound[df_secondRound['AB'] == 'AB']



#Clean INSEE code for df_baseEtabl
df_baseEtabl['Z'] = df_baseEtabl['CODGEO'].apply(lambda x: 'Z' if str(x)[0] == 'Z' else 'No')
df_baseEtabl['AB'] = df_baseEtabl['CODGEO'].apply(lambda x: 'AB' if str(x)[0:2] == '2A' or str(x)[0:2] == '2B' else 'No')
df_baseEtabl[df_baseEtabl['AB'] == 'AB']

df_baseEtabl_Z = df_baseEtabl[df_baseEtabl['Z'] == 'Z']
df_baseEtabl_AB = df_baseEtabl[df_baseEtabl['AB'] == 'AB']


#Clean INSEE code for df_salaryInfo
df_salaryInfo['Z'] = df_salaryInfo['CODGEO'].apply(lambda x: 'Z' if str(x)[0] == 'Z' else 'No')
df_salaryInfo['AB'] = df_salaryInfo['CODGEO'].apply(lambda x: 'AB' if str(x)[0:2] == '2A' or str(x)[0:2] == '2B' else 'No')
df_salaryInfo[df_salaryInfo['AB'] == 'AB']

df_salaryInfo_Z = df_salaryInfo[df_salaryInfo['Z'] == 'Z']
df_salaryInfo_AB = df_salaryInfo[df_salaryInfo['AB'] == 'AB']

In [3]:
#Clean The Data
df_geoInfo_lag =df_geoInfo[df_geoInfo['nom_commune'] == 'Laguépie'].iloc[0]
df_geoInfo_noLag = df_geoInfo[df_geoInfo['nom_commune'] != 'Laguépie']

df_geoInfo = df_geoInfo_noLag.append(df_geoInfo_lag)

#Adjust column names to match those in geoInfo
df_geoInfo = df_geoInfo.rename(index=str, columns={'nom_commune':'Commune', 'numéro_département':'Department code','numéro_circonscription':'Constituency code','code_insee':'INSEE code'})
df_geoInfo

#Adjust the Department code to match the geoInfo dataframe by droping the '0' and the beginning of the code
def removeZero(x):
    
    theList = ['00','01','02','03','04','05','06','07','08','09']
    
    if x in theList:
        return x[-1:]
    else:
        return x
    
def removeZAB(x):
    
    temp = str(x)
    
    if temp[0] == 'Z' or temp[0:2] == '2A' or temp[0:2] == '2B':
        return None
    else:
        return x

#First Round Adjustments
df_firstRound['INSEE_noZAB'] = list(map(removeZAB, df_firstRound['INSEE code']))
df_firstRound = df_firstRound.drop(['INSEE code'], axis=1)
df_firstRound = df_firstRound.rename(index=str, columns={'INSEE_noZAB':'INSEE code'})

df_firstRound = df_firstRound.dropna(subset=['INSEE code'])

df_firstRound['INSEE code'] = df_firstRound['INSEE code'].astype(int)
df_geoInfo['INSEE code'] = df_geoInfo['INSEE code'].astype(int)

df_firstRound['INSEE code'] = df_firstRound['INSEE code'].astype(str)
df_geoInfo['INSEE code'] = df_geoInfo['INSEE code'].astype(str)


#First Second Adjustments
df_secondRound['INSEE_noZAB'] = list(map(removeZAB, df_secondRound['INSEE code']))
df_secondRound = df_secondRound.drop(['INSEE code'], axis=1)
df_secondRound = df_secondRound.rename(index=str, columns={'INSEE_noZAB':'INSEE code'})

df_secondRound = df_secondRound.dropna(subset=['INSEE code'])

df_secondRound['INSEE code'] = df_secondRound['INSEE code'].astype(int)
df_secondRound['INSEE code'] = df_secondRound['INSEE code'].astype(str)

In [4]:
#Build a couple dumby datasets based on the first round data
df_townVotingData = df_firstRound
df_townCandidates = df_firstRound

#Build a couple dumby datasets based on the second round data
df_townVotingData2 = df_secondRound
df_townCandidates2 = df_secondRound

In [5]:
########################################################################################################
########################################################################################################
import re

def getMessedUp(x):
    
    x = str(x)
    newVal = x
    
    naughtyLetters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

    if x.find('.') != -1:
        newVal = x[0:x.find('.')]
    elif len(x) == 4:
        if (x[1] in naughtyLetters) or (x[-1:] in naughtyLetters):
            newVal = re.sub('[A-Z]', '', x)
    
    return newVal


#Get General voter info for the town/INSEE code Round 1

df_townVotingData['INSEE_noZAB'] = list(map(removeZAB, df_townVotingData['INSEE code']))
df_townVotingData = df_townVotingData.drop(['INSEE code'], axis=1)
df_townVotingData = df_townVotingData.rename(index=str, columns={'INSEE_noZAB':'INSEE code'})

df_townVotingData = df_townVotingData.dropna(subset=['INSEE code'])

df_townVotingData['Polling_Stations_Cleaned'] = list(map(getMessedUp, df_townVotingData['Polling station']))
df_townVotingData = df_townVotingData.drop(['Polling station'], axis=1)
df_townVotingData = df_townVotingData.rename(index=str, columns={'Polling_Stations_Cleaned': 'Polling station'})

df_townVotingData['INSEE code'] = df_townVotingData['INSEE code'].astype(int)
df_townVotingData['INSEE code'] = df_townVotingData['INSEE code'].astype(str)

df_townVotingData['Polling station'] = df_townVotingData['Polling station'].astype(int)
df_townVotingData['Polling station'] = df_townVotingData['Polling station'].astype(str)

#df_firstRound['Joiner_poll'] = df_firstRound['Commune'].astype(str) + '-' + df_firstRound['INSEE code'].astype(str) + '-' + df_firstRound['Polling station'].astype(str)
#df_firstRound = df_firstRound.drop_duplicates(subset=['Joiner_poll'])

x = df_townVotingData.groupby(['Commune','INSEE code','Polling station']).sum().reset_index()


x = x.drop(['Unnamed: 0','Constituency code','Commune code','% Abs/Reg','Polling station',
                   '% NOTA/Reg','% NOTA/Vot','% Nulls/Reg','% Nulls/Vot','% Votes/Reg',
                   '% Votes/Exp','Postal code','Signboard','% Vot/Reg','% Exp/Reg',
                   '% Exp/Vot','Voted'], axis=1)

x['Registered'] = x['Registered'].apply(lambda x: x/11)
x['Abstentions'] = x['Abstentions'].apply(lambda x: x/11)
x['Voters'] = x['Voters'].apply(lambda x: x/11)
x['None of the above(NOTA)'] = x['None of the above(NOTA)'].apply(lambda x: x/11)
x['Nulls'] = x['Nulls'].apply(lambda x: x/11)
x['Expressed'] = x['Expressed'].apply(lambda x: x/11)

df_townVotingData = x.groupby(['Commune','INSEE code']).sum().reset_index()

########################################################################################################
########################################################################################################

In [6]:
#Get General voter info for the town/INSEE code Round 2

import re

def getMessedUp(x):
    
    x = str(x)
    newVal = x
    
    naughtyLetters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

    if x.find('.') != -1:
        newVal = x[0:x.find('.')]
    elif len(x) == 4:
        if (x[1] in naughtyLetters) or (x[-1:] in naughtyLetters):
            newVal = re.sub('[A-Z]', '', x)
    
    return newVal

df_townVotingData2['INSEE_noZAB'] = list(map(removeZAB, df_townVotingData2['INSEE code']))
df_townVotingData2 = df_townVotingData2.drop(['INSEE code'], axis=1)
df_townVotingData2 = df_townVotingData2.rename(index=str, columns={'INSEE_noZAB':'INSEE code'})

df_townVotingData2 = df_townVotingData2.dropna(subset=['INSEE code'])

df_townVotingData2['Polling_Stations_Cleaned'] = list(map(getMessedUp, df_townVotingData2['Polling station']))
df_townVotingData2 = df_townVotingData2.drop(['Polling station'], axis=1)
df_townVotingData2 = df_townVotingData2.rename(index=str, columns={'Polling_Stations_Cleaned': 'Polling station'})

df_townVotingData2['INSEE code'] = df_townVotingData2['INSEE code'].astype(int)
df_townVotingData2['INSEE code'] = df_townVotingData2['INSEE code'].astype(str)

df_townVotingData2['Polling station'] = df_townVotingData2['Polling station'].astype(int)
df_townVotingData2['Polling station'] = df_townVotingData2['Polling station'].astype(str)

#df_firstRound['Joiner_poll'] = df_firstRound['Commune'].astype(str) + '-' + df_firstRound['INSEE code'].astype(str) + '-' + df_firstRound['Polling station'].astype(str)
#df_firstRound = df_firstRound.drop_duplicates(subset=['Joiner_poll'])

x2 = df_townVotingData2.groupby(['Commune','INSEE code','Polling station']).sum().reset_index()


x2 = x2.drop(['Unnamed: 0','Constituency code','Commune code','% Abs/Reg','Polling station',
                   '% NOTA/Reg','% NOTA/Vot','% Nulls/Reg','% Nulls/Vot','% Votes/Reg',
                   '% Votes/Exp','Postal code','Signboard','% Vot/Reg','% Exp/Reg',
                   '% Exp/Vot','Voted'], axis=1)

x2['Registered'] = x2['Registered'].apply(lambda x: x/2)
x2['Abstentions'] = x2['Abstentions'].apply(lambda x: x/2)
x2['Voters'] = x2['Voters'].apply(lambda x: x/2)
x2['None of the above(NOTA)'] = x2['None of the above(NOTA)'].apply(lambda x: x/2)
x2['Nulls'] = x2['Nulls'].apply(lambda x: x/2)
x2['Expressed'] = x2['Expressed'].apply(lambda x: x/2)

df_townVotingData2 = x2.groupby(['Commune','INSEE code']).sum().reset_index()



In [7]:
#Lets get the votes by candidate by town/INSEE code along with total votes and winner for
#round 1
df_townCandidates['INSEE_noZAB'] = list(map(removeZAB, df_townCandidates['INSEE code']))
df_townCandidates = df_townCandidates.drop(['INSEE code'], axis=1)
df_townCandidates = df_townCandidates.rename(index=str, columns={'INSEE_noZAB':'INSEE code'})

df_townCandidates = df_townCandidates.dropna(subset=['INSEE code'])

df_townCandidates['INSEE code'] = df_townCandidates['INSEE code'].astype(int)
df_townCandidates['INSEE code'] = df_townCandidates['INSEE code'].astype(str)

df_townCandidates['Department code'] = df_townCandidates['Department code'].astype(int)
df_townCandidates['Department code'] = df_townCandidates['Department code'].astype(str)

# .str.lower()
df_townCandidates['Commune'] = df_townCandidates['Commune'].apply(lambda x: str(x).lower())

############################################################################################################
df_departmentData = df_townCandidates.drop(['Unnamed: 0','Department','Constituency code',
                        'Constituency','Commune code','Registered','Abstentions','% Abs/Reg',
                        'Voters','% Vot/Reg','None of the above(NOTA)','% NOTA/Reg',
                        '% NOTA/Vot','Nulls','% Nulls/Reg','% Nulls/Vot','Expressed',
                        '% Exp/Reg','% Exp/Vot','Signboard','Sex','First name','% Votes/Reg',
                        '% Votes/Exp','Coordinates','Polling station name','Address',
                        'Postal code','City','Poll.St.-unique','Z','AB','Polling station','Commune',
                        'INSEE code'], axis=1)

y = df_departmentData.groupby(['Department code','Surname']).sum().reset_index().sort_values(by=['Department code','Voted'], ascending=False)

############################################################################################################


df_townCandidates = df_townCandidates.drop(['Unnamed: 0','Department code','Department','Constituency code',
                        'Constituency','Commune code','Registered','Abstentions','% Abs/Reg',
                        'Voters','% Vot/Reg','None of the above(NOTA)','% NOTA/Reg',
                        '% NOTA/Vot','Nulls','% Nulls/Reg','% Nulls/Vot','Expressed',
                        '% Exp/Reg','% Exp/Vot','Signboard','Sex','First name','% Votes/Reg',
                        '% Votes/Exp','Coordinates','Polling station name','Address',
                        'Postal code','City','Poll.St.-unique','Z','AB','Polling station'], axis=1)

x = df_townCandidates.groupby(['Commune','Surname','INSEE code']).sum().reset_index()
x2 = df_townCandidates.groupby(['Commune','Surname','INSEE code']).sum().reset_index()

In [8]:
#Get winners by department
df_departmentWinners = y.drop_duplicates(subset=['Department code'], keep='first')
df_departmentWinners = df_departmentWinners.drop(['Voted'], axis=1)
df_departmentWinners = df_departmentWinners.rename(index=str, columns={'Surname':'R1 Department Winner'})
df_departmentWinners.to_csv('df_departmentWinners.csv')

In [9]:
#Get margin of victory by town/INSEE code
x2['Joiner'] = x2['Commune'].astype(str) + '-' + x2['INSEE code'].astype(str)
x2 = x2.sort_values(by=['Joiner','Voted'], ascending=False)
x2.head(20)


df_secondPlace = (x2.groupby('Joiner', as_index=False).apply(lambda x: x if len(x)==1 else x.iloc[[1]])
   .reset_index(level=0, drop=True))
df_secondPlace = df_secondPlace.rename(index=str, columns={'Voted':'SecondPlace_Votes'})
df_secondPlace = df_secondPlace.drop(['Commune','Surname','INSEE code'], axis=1)


df_firstPlace = (x2.groupby('Joiner', as_index=False).apply(lambda x: x if len(x)==1 else x.iloc[[0]])
   .reset_index(level=0, drop=True))
df_firstPlace = df_firstPlace.rename(index=str, columns={'Voted':'FirstPlace_Votes'})
df_firstPlace = df_firstPlace.drop(['Commune','Surname','INSEE code'], axis=1)

df_voteDiff = pd.merge(df_firstPlace, df_secondPlace, how='left', on='Joiner')

def getDifference(x,y):
    return x - y


df_voteDiff['FirstSecond_Diff'] = list(map(getDifference,df_voteDiff['FirstPlace_Votes'],df_voteDiff['SecondPlace_Votes']))
df_voteDiff.head()

,FirstPlace_Votes,Joiner,SecondPlace_Votes,FirstSecond_Diff
0,39,aast-64001,28,11
1,64,abainville-55001,49,15
2,94,abancourt-59001,67,27
3,183,abancourt-60001,70,113
4,86,abaucourt-54001,48,38


In [10]:
df_voteDiff.to_csv('df_voteDiff.csv')

In [11]:
#Get total votes and winner of town by town/INSEE code
x['Joiner'] = x['Commune'].astype(str) + '-' + x['INSEE code'].astype(str)
x = x.sort_values(by=['Joiner','Voted'], ascending=False)
df_townWinner = x.drop_duplicates(subset=['Joiner'], keep='first')
df_townWinner = df_townWinner.rename(index=str, columns={'Surname':'Winner'})
df_townWinner = df_townWinner.drop(['Voted','Commune','INSEE code'], axis=1)
df_townWinner

#Convert votes for candidates into columns
x = df_townCandidates.groupby(['Commune','INSEE code','Surname']).sum().unstack().reset_index()
x.columns = x.columns.droplevel()
x.columns.values[0] = 'Commune'
x.columns.values[1] = 'INSEE code'

df_townCandidates = x

In [12]:
######################################################################################

#Lets do all the above for Round 2:

df_townCandidates2['INSEE_noZAB'] = list(map(removeZAB, df_townCandidates2['INSEE code']))
df_townCandidates2 = df_townCandidates2.drop(['INSEE code'], axis=1)
df_townCandidates2 = df_townCandidates2.rename(index=str, columns={'INSEE_noZAB':'INSEE code'})

df_townCandidates2 = df_townCandidates2.dropna(subset=['INSEE code'])

df_townCandidates2['INSEE code'] = df_townCandidates2['INSEE code'].astype(int)
df_townCandidates2['INSEE code'] = df_townCandidates2['INSEE code'].astype(str)

# .str.lower()
df_townCandidates2['Commune'] = df_townCandidates2['Commune'].apply(lambda x: str(x).lower())

df_townCandidates2 = df_townCandidates2.drop(['Unnamed: 0','Department code','Department','Constituency code',
                        'Constituency','Commune code','Registered','Abstentions','% Abs/Reg',
                        'Voters','% Vot/Reg','None of the above(NOTA)','% NOTA/Reg',
                        '% NOTA/Vot','Nulls','% Nulls/Reg','% Nulls/Vot','Expressed',
                        '% Exp/Reg','% Exp/Vot','Signboard','Sex','First name','% Votes/Reg',
                        '% Votes/Exp','Coordinates','Polling station name','Address',
                        'Postal code','City','Poll.St.-unique','Z','AB','Polling station'], axis=1)

x2 = df_townCandidates2.groupby(['Commune','Surname','INSEE code']).sum().reset_index()

#Get total votes and winner of town by town/INSEE code
x2['Joiner'] = x2['Commune'].astype(str) + '-' + x2['INSEE code'].astype(str)
x2 = x2.sort_values(by=['Joiner','Voted'], ascending=False)
df_townWinner2 = x2.drop_duplicates(subset=['Joiner'], keep='first')
df_townWinner2 = df_townWinner2.rename(index=str, columns={'Surname':'Winner'})
df_townWinner2 = df_townWinner2.drop(['Voted','Commune','INSEE code'], axis=1)
df_townWinner2

#Convert votes for candidates into columns
x2 = df_townCandidates2.groupby(['Commune','INSEE code','Surname']).sum().unstack().reset_index()
x2.columns = x2.columns.droplevel()
x2.columns.values[0] = 'Commune'
x2.columns.values[1] = 'INSEE code'

df_townCandidates2 = x2

#####################################################################################

In [13]:
df_geoInfo['Department_Code_Adjusted'] = list(map(removeZero, df_geoInfo['Department code']))
df_geoInfo = df_geoInfo.drop(['Department code'], axis=1)
df_geoInfo = df_geoInfo.rename(index=str, columns={'Department_Code_Adjusted':'Department code'})

df_firstRound['Commune'] = df_firstRound['Commune'].apply(lambda x: str(x).lower())
df_geoInfo['Commune'] = df_geoInfo['Commune'].apply(lambda x: str(x).lower())
df_townVotingData['Commune'] = df_townVotingData['Commune'].apply(lambda x: str(x).lower())

df_secondRound['Commune'] = df_secondRound['Commune'].apply(lambda x: str(x).lower())
df_townVotingData2['Commune'] = df_townVotingData2['Commune'].apply(lambda x: str(x).lower())

In [14]:
#Join FirstRound,geo, voter, and candidates data together:

df_geoInfo['Joiner'] = df_geoInfo['Commune'].astype(str) + '-' + df_geoInfo['INSEE code'].astype(str)
df_firstRound['Joiner'] = df_firstRound['Commune'].astype(str) + '-' + df_firstRound['INSEE code'].astype(str)
df_townVotingData['Joiner'] = df_townVotingData['Commune'].astype(str) + '-' + df_townVotingData['INSEE code'].astype(str)
df_townCandidates['Joiner'] = df_townCandidates['Commune'].astype(str) + '-' + df_townCandidates['INSEE code'].astype(str)

df_secondRound['Joiner'] = df_secondRound['Commune'].astype(str) + '-' + df_secondRound['INSEE code'].astype(str)
df_townVotingData2['Joiner'] = df_townVotingData2['Commune'].astype(str) + '-' + df_townVotingData2['INSEE code'].astype(str)
df_townCandidates2['Joiner'] = df_townCandidates2['Commune'].astype(str) + '-' + df_townCandidates2['INSEE code'].astype(str)

df_geoInfo = df_geoInfo.drop_duplicates(subset=['Joiner'])

In [15]:
temp1 = pd.merge(df_townWinner, df_geoInfo, how='left', on='Joiner')
temp2 = pd.merge(temp1, df_townVotingData, how='left', on='Joiner')
df_BigJoin_round1 = pd.merge(temp2, df_townCandidates, how='left', on='Joiner')

df_BigJoin_round1 = df_BigJoin_round1.drop(['Commune_y','INSEE code_y'], axis=1)
df_BigJoin_round1 = df_BigJoin_round1.rename(index=str, columns={'Commune_x':'Commune','INSEE code_x':'INSEE code'})

In [16]:
#df_BigJoin_round1.dtypes
df_BigJoin_round1.head()

,Winner,Joiner,EU_circo,code_région,nom_région,chef.lieu_région,nom_département,préfecture,Constituency code,Commune,...,ASSELINEAU,CHEMINADE,DUPONT-AIGNAN,FILLON,HAMON,LASSALLE,LE PEN,MACRON,MÉLENCHON,POUTOU
0,LE PEN,zuytpeene-59669,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Nord,Lille,14.0,zuytpeene,...,1,0,34,74,19,3,93,73,60,6
1,LE PEN,zuydcoote-59668,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Nord,Lille,14.0,zuydcoote,...,7,6,75,146,77,14,271,210,161,14
2,LE PEN,zutkerque-62906,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Pas-de-Calais,Arras,7.0,zutkerque,...,5,1,92,182,34,3,400,168,150,1
3,FILLON,zudausques-62905,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Pas-de-Calais,Arras,3.0,zudausques,...,3,1,44,137,29,6,113,133,114,7
4,MACRON,zoufftgen-57764,Est,41.0,Lorraine,Metz,Moselle,Metz,9.0,zoufftgen,...,5,1,25,117,34,10,139,196,71,7


In [17]:
temp1 = pd.merge(df_townWinner2, df_geoInfo, how='left', on='Joiner')
temp2 = pd.merge(temp1, df_townVotingData2, how='left', on='Joiner')
df_BigJoin_round2 = pd.merge(temp2, df_townCandidates2, how='left', on='Joiner')

df_BigJoin_round2 = df_BigJoin_round2.drop(['Commune_y','INSEE code_y'], axis=1)
df_BigJoin_round2 = df_BigJoin_round2.rename(index=str, columns={'Commune_x':'Commune','INSEE code_x':'INSEE code'})

In [18]:
df_BigJoin_round1.dtypes

Winner                      object
Joiner                      object
EU_circo                    object
code_région                float64
nom_région                  object
chef.lieu_région            object
nom_département             object
préfecture                  object
Constituency code          float64
Commune                     object
codes_postaux               object
INSEE code                  object
latitude                   float64
longitude                   object
éloignement                float64
Department code             object
Registered                 float64
Abstentions                float64
Voters                     float64
None of the above(NOTA)    float64
Nulls                      float64
Expressed                  float64
Commune                     object
INSEE code                  object
ARTHAUD                      int64
ASSELINEAU                   int64
CHEMINADE                    int64
DUPONT-AIGNAN                int64
FILLON              

In [19]:
df_BigJoin_round2.dtypes

Winner                      object
Joiner                      object
EU_circo                    object
code_région                float64
nom_région                  object
chef.lieu_région            object
nom_département             object
préfecture                  object
Constituency code          float64
Commune                     object
codes_postaux               object
INSEE code                  object
latitude                   float64
longitude                   object
éloignement                float64
Department code             object
Registered                 float64
Abstentions                float64
Voters                     float64
None of the above(NOTA)    float64
Nulls                      float64
Expressed                  float64
Commune                     object
INSEE code                  object
LE PEN                       int64
MACRON                       int64
dtype: object

In [20]:
#Lets format the population dataset!!!
df_population = df_population.rename(index = str, columns={'LIBGEO':'Commune', 'CODGEO':'INSEE code'})

In [21]:
df_population.AGEQ80_17 = df_population.AGEQ80_17.astype(str)
df_population.SEXE = df_population.SEXE.astype(str)
df_population.MOCO = df_population.MOCO.astype(str) 

In [22]:
def ageAdjust(x):
    
    returnVal = x
    
    theDict = {'0':'Under 20', 
             '5': 'Under 20', 
             '10':'Under 20', 
             '15': 'Under 20', 
             '20':'20-29', 
             '25':'20-29', 
             '30':'30-39',
             '35':'30-39',
             '40':'40-49',
             '45':'40-49',
             '50':'50-59',
             '55':'50-59',
             '60':'60-69',
             '65':'60-69',
             '70':'70-79',
             '75':'70-79',
             '80':'80+'}
    
    if x in theDict:
        returnVal = theDict[x]
    
    return returnVal


def createDemo(x, label, pop):
    
    if x == label:
        return pop
    else:
        return 0
    
        


#Breakout Population by Age, Household, and Gender

#Household
df_population['Commune'] = df_population['Commune'].apply(lambda x: str(x).lower())
df_population['Age'] = list(map(ageAdjust, df_population['AGEQ80_17']))

df_population['11'] = df_population['MOCO'].apply(lambda x: '11' if x == '11' else 'no')
df_population['Children_2Parents'] = list(map(createDemo, df_population['MOCO'], df_population['11'], df_population['NB']))

df_population['12'] = df_population['MOCO'].apply(lambda x: '12' if x == '12' else 'no')
df_population['Children_1Parent'] = list(map(createDemo, df_population['MOCO'], df_population['12'], df_population['NB']))

df_population['21'] = df_population['MOCO'].apply(lambda x: '21' if x == '21' else 'no')
df_population['NoChildren_Couple'] = list(map(createDemo, df_population['MOCO'], df_population['21'], df_population['NB']))

df_population['22'] = df_population['MOCO'].apply(lambda x: '22' if x == '22' else 'no')
df_population['Children_Couple'] = list(map(createDemo, df_population['MOCO'], df_population['22'], df_population['NB']))

df_population['23'] = df_population['MOCO'].apply(lambda x: '23' if x == '23' else 'no')
df_population['Children_Single'] = list(map(createDemo, df_population['MOCO'], df_population['23'], df_population['NB']))

df_population['31'] = df_population['MOCO'].apply(lambda x: '31' if x == '31' else 'no')
df_population['NoChildren_Adults'] = list(map(createDemo, df_population['MOCO'], df_population['31'], df_population['NB']))

df_population['32'] = df_population['MOCO'].apply(lambda x: '32' if x == '32' else 'no')
df_population['NoChildren_Alone'] = list(map(createDemo, df_population['MOCO'], df_population['32'], df_population['NB']))


#Gender
df_population['Male'] = df_population['SEXE'].apply(lambda x: '1' if x == '1' else 0)
df_population['Male_pop'] = list(map(createDemo, df_population['SEXE'], df_population['Male'], df_population['NB']))

df_population['Female'] = df_population['SEXE'].apply(lambda x: '2' if x == '2' else 0)
df_population['Female_pop'] = list(map(createDemo, df_population['SEXE'], df_population['Female'], df_population['NB']))


#Age
df_population['Under_20'] = df_population['Age'].apply(lambda x: 'Under 20' if x == 'Under 20' else 0)
df_population['Under_20_pop'] = list(map(createDemo, df_population['Age'], df_population['Under_20'], df_population['NB']))

df_population['20-29'] = df_population['Age'].apply(lambda x: '20-29' if x == '20-29' else 0)
df_population['20-29_pop'] = list(map(createDemo, df_population['Age'], df_population['20-29'], df_population['NB']))

df_population['30-39'] = df_population['Age'].apply(lambda x: '30-39' if x == '30-39' else 0)
df_population['30-39_pop'] = list(map(createDemo, df_population['Age'], df_population['30-39'], df_population['NB']))

df_population['40-49'] = df_population['Age'].apply(lambda x: '40-49' if x == '40-49' else 0)
df_population['40-49_pop'] = list(map(createDemo, df_population['Age'], df_population['40-49'], df_population['NB']))

df_population['50-59'] = df_population['Age'].apply(lambda x: '50-59' if x == '50-59' else 0)
df_population['50-59_pop'] = list(map(createDemo, df_population['Age'], df_population['50-59'], df_population['NB']))

df_population['60-69'] = df_population['Age'].apply(lambda x: '60-69' if x == '60-69' else 0)
df_population['60-69_pop'] = list(map(createDemo, df_population['Age'], df_population['60-69'], df_population['NB']))

df_population['70-79'] = df_population['Age'].apply(lambda x: '70-79' if x == '70-79' else 0)
df_population['70-79_pop'] = list(map(createDemo, df_population['Age'], df_population['70-79'], df_population['NB']))

df_population['80+'] = df_population['Age'].apply(lambda x: '80+' if x == '80+' else 0)
df_population['80Plus_pop'] = list(map(createDemo, df_population['Age'], df_population['80+'], df_population['NB']))

#df_population['Joiner'] = df_population['Commune'].astype(str) + '-' + df_population['INSEE code'].astype(str)

In [23]:
#df_population.to_csv('population_check.csv')

In [24]:
#cat_fields = ['Age','Gender','Household']
#df_population = pd.get_dummies(df_population, columns = cat_fields, drop_first=True)

In [25]:
df_population = df_population.drop(['NIVGEO','MOCO','AGEQ80_17','SEXE','Age','11','12','21','22','23','31','32','Male','Female','Under_20','20-29','30-39','40-49','50-59','60-69','70-79','80+'], axis=1)
df_population.dtypes

INSEE code           object
Commune              object
NB                    int64
Children_2Parents     int64
Children_1Parent      int64
NoChildren_Couple     int64
Children_Couple       int64
Children_Single       int64
NoChildren_Adults     int64
NoChildren_Alone      int64
Male_pop              int64
Female_pop            int64
Under_20_pop          int64
20-29_pop             int64
30-39_pop             int64
40-49_pop             int64
50-59_pop             int64
60-69_pop             int64
70-79_pop             int64
80Plus_pop            int64
dtype: object

In [26]:
x = df_population.groupby(['Commune','INSEE code']).sum().reset_index()

In [27]:
x['Joiner'] = x['Commune'].astype(str) + '-' + x['INSEE code'].astype(str)
df_population_formatted = x.rename(index=str, columns = {'NB':'Total_Population'})

In [28]:
#Save the formatted population info as a csv so we don't have to rerun everything
#df_population_formatted.to_csv('df_population_formatted.csv')

df_population_formatted.head()

,Commune,INSEE code,Total_Population,Children_2Parents,Children_1Parent,NoChildren_Couple,Children_Couple,Children_Single,NoChildren_Adults,NoChildren_Alone,...,Female_pop,Under_20_pop,20-29_pop,30-39_pop,40-49_pop,50-59_pop,60-69_pop,70-79_pop,80Plus_pop,Joiner
0,aast,64001,144,41,0,27,53,0,7,16,...,70,35,0,24,33,24,12,16,0,aast-64001
1,abainville,55001,272,80,0,80,88,0,0,24,...,133,67,21,26,21,48,69,8,12,abainville-55001
2,abancourt,59001,436,120,20,96,136,16,16,32,...,220,108,56,36,84,72,12,44,24,abancourt-59001
3,abancourt,60001,592,176,12,160,179,12,4,49,...,295,161,106,99,43,65,66,39,13,abancourt-60001
4,abaucourt,54001,290,50,35,80,60,25,0,40,...,150,75,15,55,35,20,50,15,25,abaucourt-54001


In [29]:
#Lets Join the data sets!!!
df_BigJoin_round1_wPop = pd.merge(df_BigJoin_round1, df_population_formatted, how='left', on='Joiner')
df_BigJoin_round2_wPop = pd.merge(df_BigJoin_round2, df_population_formatted, how='left', on='Joiner')

In [30]:
df_BigJoin_round1_wPop

,Winner,Joiner,EU_circo,code_région,nom_région,chef.lieu_région,nom_département,préfecture,Constituency code,Commune_x,...,Male_pop,Female_pop,Under_20_pop,20-29_pop,30-39_pop,40-49_pop,50-59_pop,60-69_pop,70-79_pop,80Plus_pop
0,LE PEN,zuytpeene-59669,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Nord,Lille,14.0,zuytpeene,...,242.0,229.0,145.0,48.0,84.0,69.0,59.0,27.0,30.0,9.0
1,LE PEN,zuydcoote-59668,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Nord,Lille,14.0,zuydcoote,...,756.0,788.0,404.0,120.0,136.0,272.0,276.0,200.0,92.0,44.0
2,LE PEN,zutkerque-62906,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Pas-de-Calais,Arras,7.0,zutkerque,...,770.0,855.0,406.0,231.0,168.0,223.0,274.0,226.0,58.0,39.0
3,FILLON,zudausques-62905,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Pas-de-Calais,Arras,3.0,zudausques,...,452.0,388.0,272.0,72.0,136.0,116.0,132.0,84.0,24.0,4.0
4,MACRON,zoufftgen-57764,Est,41.0,Lorraine,Metz,Moselle,Metz,9.0,zoufftgen,...,531.0,508.0,286.0,82.0,309.0,145.0,76.0,78.0,47.0,16.0
5,LE PEN,zouafques-62904,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Pas-de-Calais,Arras,7.0,zouafques,...,316.0,312.0,216.0,100.0,112.0,104.0,24.0,44.0,16.0,12.0
6,LE PEN,zoteux-62903,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Pas-de-Calais,Arras,3.0,zoteux,...,288.0,239.0,158.0,67.0,85.0,58.0,73.0,44.0,26.0,16.0
7,MACRON,zommange-57763,Est,41.0,Lorraine,Metz,Moselle,Metz,4.0,zommange,...,20.0,12.0,0.0,4.0,0.0,4.0,12.0,4.0,8.0,0.0
8,MACRON,zoebersdorf-67560,Est,42.0,Alsace,Strasbourg,Bas-Rhin,Strasbourg,7.0,zoebersdorf,...,92.0,68.0,24.0,16.0,16.0,20.0,44.0,12.0,16.0,12.0
9,LE PEN,zittersheim-67559,Est,42.0,Alsace,Strasbourg,Bas-Rhin,Strasbourg,7.0,zittersheim,...,93.0,110.0,37.0,13.0,32.0,23.0,20.0,45.0,17.0,16.0


In [31]:
df_BigJoin_round2_wPop

,Winner,Joiner,EU_circo,code_région,nom_région,chef.lieu_région,nom_département,préfecture,Constituency code,Commune_x,...,Male_pop,Female_pop,Under_20_pop,20-29_pop,30-39_pop,40-49_pop,50-59_pop,60-69_pop,70-79_pop,80Plus_pop
0,MACRON,zuytpeene-59669,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Nord,Lille,14.0,zuytpeene,...,242.0,229.0,145.0,48.0,84.0,69.0,59.0,27.0,30.0,9.0
1,MACRON,zuydcoote-59668,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Nord,Lille,14.0,zuydcoote,...,756.0,788.0,404.0,120.0,136.0,272.0,276.0,200.0,92.0,44.0
2,LE PEN,zutkerque-62906,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Pas-de-Calais,Arras,7.0,zutkerque,...,770.0,855.0,406.0,231.0,168.0,223.0,274.0,226.0,58.0,39.0
3,MACRON,zudausques-62905,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Pas-de-Calais,Arras,3.0,zudausques,...,452.0,388.0,272.0,72.0,136.0,116.0,132.0,84.0,24.0,4.0
4,MACRON,zoufftgen-57764,Est,41.0,Lorraine,Metz,Moselle,Metz,9.0,zoufftgen,...,531.0,508.0,286.0,82.0,309.0,145.0,76.0,78.0,47.0,16.0
5,LE PEN,zouafques-62904,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Pas-de-Calais,Arras,7.0,zouafques,...,316.0,312.0,216.0,100.0,112.0,104.0,24.0,44.0,16.0,12.0
6,LE PEN,zoteux-62903,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Pas-de-Calais,Arras,3.0,zoteux,...,288.0,239.0,158.0,67.0,85.0,58.0,73.0,44.0,26.0,16.0
7,MACRON,zommange-57763,Est,41.0,Lorraine,Metz,Moselle,Metz,4.0,zommange,...,20.0,12.0,0.0,4.0,0.0,4.0,12.0,4.0,8.0,0.0
8,MACRON,zoebersdorf-67560,Est,42.0,Alsace,Strasbourg,Bas-Rhin,Strasbourg,7.0,zoebersdorf,...,92.0,68.0,24.0,16.0,16.0,20.0,44.0,12.0,16.0,12.0
9,MACRON,zittersheim-67559,Est,42.0,Alsace,Strasbourg,Bas-Rhin,Strasbourg,7.0,zittersheim,...,93.0,110.0,37.0,13.0,32.0,23.0,20.0,45.0,17.0,16.0


In [32]:
#Lets format and add company data

df_baseEtabl['INSEE_noZAB'] = list(map(removeZAB, df_baseEtabl['CODGEO']))
df_baseEtabl = df_baseEtabl.drop(['CODGEO'], axis=1)
df_baseEtabl = df_baseEtabl.rename(index=str, columns={'INSEE_noZAB':'CODGEO'})
df_baseEtabl = df_baseEtabl.dropna(subset=['CODGEO'])

df_baseEtabl['LIBGEO'] = df_baseEtabl['LIBGEO'].apply(lambda x: str(x).lower())
df_baseEtabl['CODGEO'] = df_baseEtabl['CODGEO'].astype(int)
df_baseEtabl['CODGEO'] = df_baseEtabl['CODGEO'].astype(str)

df_baseEtabl['Joiner'] = df_baseEtabl['LIBGEO'].astype(str) + '-' + df_baseEtabl['CODGEO'].astype(str)

df_baseEtabl_formatted = df_baseEtabl.drop(['REG','LIBGEO','DEP','CODGEO','Z','AB'], axis=1)

In [67]:
df_firstRound_final = pd.merge(df_BigJoin_round1_wPop, df_baseEtabl_formatted, how='left', on='Joiner')
df_firstRound_final = df_firstRound_final.drop(['Commune_y','INSEE code_y'], axis=1)

df_secondRound_final = pd.merge(df_BigJoin_round2_wPop, df_baseEtabl_formatted, how='left', on='Joiner')
df_secondRound_final = df_secondRound_final.drop(['Commune_y','INSEE code_y'], axis=1)

In [68]:
df_secondRound_final

,Winner,Joiner,EU_circo,code_région,nom_région,chef.lieu_région,nom_département,préfecture,Constituency code,Commune_x,...,E14TST,E14TS0ND,E14TS1,E14TS6,E14TS10,E14TS20,E14TS50,E14TS100,E14TS200,E14TS500
0,MACRON,zuytpeene-59669,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Nord,Lille,14.0,zuytpeene,...,11.0,6.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,MACRON,zuydcoote-59668,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Nord,Lille,14.0,zuydcoote,...,44.0,32.0,10.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,LE PEN,zutkerque-62906,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Pas-de-Calais,Arras,7.0,zutkerque,...,45.0,37.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,MACRON,zudausques-62905,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Pas-de-Calais,Arras,3.0,zudausques,...,28.0,18.0,7.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,MACRON,zoufftgen-57764,Est,41.0,Lorraine,Metz,Moselle,Metz,9.0,zoufftgen,...,28.0,24.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,LE PEN,zouafques-62904,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Pas-de-Calais,Arras,7.0,zouafques,...,23.0,16.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
6,LE PEN,zoteux-62903,Nord-Ouest,31.0,Nord-Pas-de-Calais,Lille,Pas-de-Calais,Arras,3.0,zoteux,...,23.0,11.0,8.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0
7,MACRON,zommange-57763,Est,41.0,Lorraine,Metz,Moselle,Metz,4.0,zommange,...,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,MACRON,zoebersdorf-67560,Est,42.0,Alsace,Strasbourg,Bas-Rhin,Strasbourg,7.0,zoebersdorf,...,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,MACRON,zittersheim-67559,Est,42.0,Alsace,Strasbourg,Bas-Rhin,Strasbourg,7.0,zittersheim,...,9.0,8.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
#Lets ad salary data

df_salaryInfo['INSEE_noZAB'] = list(map(removeZAB, df_salaryInfo['CODGEO']))
df_salaryInfo = df_salaryInfo.drop(['CODGEO'], axis=1)
df_salaryInfo = df_salaryInfo.rename(index=str, columns={'INSEE_noZAB':'CODGEO'})
df_salaryInfo = df_salaryInfo.dropna(subset=['CODGEO'])

df_salaryInfo['LIBGEO'] = df_salaryInfo['LIBGEO'].apply(lambda x: str(x).lower())
df_salaryInfo['CODGEO'] = df_salaryInfo['CODGEO'].astype(int)
df_salaryInfo['CODGEO'] = df_salaryInfo['CODGEO'].astype(str)

df_salaryInfo['Joiner'] = df_salaryInfo['LIBGEO'].astype(str) + '-' + df_salaryInfo['CODGEO'].astype(str)

df_salaryInfo_formatted = df_salaryInfo.drop(['LIBGEO','CODGEO','Z','AB'], axis=1)

In [70]:
#LETS FINALIZE THIS SHIT!!!!!!!!

#Gender ratio:
def getRatio(male, female):
    return (male/female)

df_firstRound_final['genderRatio'] = list(map(getRatio,df_firstRound_final['Male_pop'],df_firstRound_final['Female_pop']))


#Get Age Proportions:
df_firstRound_final['Age_Under20_prop'] = list(map(getRatio,df_firstRound_final['Under_20_pop'],df_firstRound_final['Total_Population']))
df_firstRound_final['Age_20-29_prop'] = list(map(getRatio,df_firstRound_final['20-29_pop'],df_firstRound_final['Total_Population']))
df_firstRound_final['Age_30-39_prop'] = list(map(getRatio,df_firstRound_final['30-39_pop'],df_firstRound_final['Total_Population']))
df_firstRound_final['Age_40-49_prop'] = list(map(getRatio,df_firstRound_final['40-49_pop'],df_firstRound_final['Total_Population']))
df_firstRound_final['Age_50-59_prop'] = list(map(getRatio,df_firstRound_final['50-59_pop'],df_firstRound_final['Total_Population']))
df_firstRound_final['Age_60-69_prop'] = list(map(getRatio,df_firstRound_final['60-69_pop'],df_firstRound_final['Total_Population']))
df_firstRound_final['Age_70-79_prop'] = list(map(getRatio,df_firstRound_final['70-79_pop'],df_firstRound_final['Total_Population']))
df_firstRound_final['Age_80Plus_prop'] = list(map(getRatio,df_firstRound_final['80Plus_pop'],df_firstRound_final['Total_Population']))


#Get HH proportions:
df_firstRound_final['Children_2Parents_prop'] = list(map(getRatio,df_firstRound_final['Children_2Parents'],df_firstRound_final['Total_Population']))
df_firstRound_final['Children_1Parent_prop'] = list(map(getRatio,df_firstRound_final['Children_1Parent'],df_firstRound_final['Total_Population']))
df_firstRound_final['NoChildren_Couple_prop'] = list(map(getRatio,df_firstRound_final['NoChildren_Couple'],df_firstRound_final['Total_Population']))
df_firstRound_final['Children_Couple_prop'] = list(map(getRatio,df_firstRound_final['Children_Couple'],df_firstRound_final['Total_Population']))
df_firstRound_final['Children_Single_prop'] = list(map(getRatio,df_firstRound_final['Children_Single'],df_firstRound_final['Total_Population']))
df_firstRound_final['NoChildren_Adults_prop'] = list(map(getRatio,df_firstRound_final['NoChildren_Adults'],df_firstRound_final['Total_Population']))
df_firstRound_final['NoChildren_Alone_prop'] = list(map(getRatio,df_firstRound_final['NoChildren_Alone'],df_firstRound_final['Total_Population']))


#Get Voter Ratios:
df_firstRound_final['% Abs/Reg'] = list(map(getRatio,df_firstRound_final['Abstentions'],df_firstRound_final['Registered']))
df_firstRound_final['% Voter/Reg'] = list(map(getRatio,df_firstRound_final['Voters'],df_firstRound_final['Registered']))
df_firstRound_final['% NOTA/Reg'] = list(map(getRatio,df_firstRound_final['None of the above(NOTA)'],df_firstRound_final['Registered']))
df_firstRound_final['% NOTA/Voter'] = list(map(getRatio,df_firstRound_final['None of the above(NOTA)'],df_firstRound_final['Voters']))
df_firstRound_final['% Nulls/Reg'] = list(map(getRatio,df_firstRound_final['Nulls'],df_firstRound_final['Registered']))
df_firstRound_final['% Nulls/Voter'] = list(map(getRatio,df_firstRound_final['Nulls'],df_firstRound_final['Voters']))
df_firstRound_final['% Exp/Reg'] = list(map(getRatio,df_firstRound_final['Expressed'],df_firstRound_final['Registered']))
df_firstRound_final['% Exp/Voter'] = list(map(getRatio,df_firstRound_final['Expressed'],df_firstRound_final['Voters']))


#############################################################################################

#Now for second round

#Get Age Proportions:
df_secondRound_final['Age_Under20_prop'] = list(map(getRatio,df_secondRound_final['Under_20_pop'],df_secondRound_final['Total_Population']))
df_secondRound_final['Age_20-29_prop'] = list(map(getRatio,df_secondRound_final['20-29_pop'],df_secondRound_final['Total_Population']))
df_secondRound_final['Age_30-39_prop'] = list(map(getRatio,df_secondRound_final['30-39_pop'],df_secondRound_final['Total_Population']))
df_secondRound_final['Age_40-49_prop'] = list(map(getRatio,df_secondRound_final['40-49_pop'],df_secondRound_final['Total_Population']))
df_secondRound_final['Age_50-59_prop'] = list(map(getRatio,df_secondRound_final['50-59_pop'],df_secondRound_final['Total_Population']))
df_secondRound_final['Age_60-69_prop'] = list(map(getRatio,df_secondRound_final['60-69_pop'],df_secondRound_final['Total_Population']))
df_secondRound_final['Age_70-79_prop'] = list(map(getRatio,df_secondRound_final['70-79_pop'],df_secondRound_final['Total_Population']))
df_secondRound_final['Age_80Plus_prop'] = list(map(getRatio,df_secondRound_final['80Plus_pop'],df_secondRound_final['Total_Population']))


#Get HH proportions:
df_secondRound_final['Children_2Parents_prop'] = list(map(getRatio,df_secondRound_final['Children_2Parents'],df_secondRound_final['Total_Population']))
df_secondRound_final['Children_1Parent_prop'] = list(map(getRatio,df_secondRound_final['Children_1Parent'],df_secondRound_final['Total_Population']))
df_secondRound_final['NoChildren_Couple_prop'] = list(map(getRatio,df_secondRound_final['NoChildren_Couple'],df_secondRound_final['Total_Population']))
df_secondRound_final['Children_Couple_prop'] = list(map(getRatio,df_secondRound_final['Children_Couple'],df_secondRound_final['Total_Population']))
df_secondRound_final['Children_Single_prop'] = list(map(getRatio,df_secondRound_final['Children_Single'],df_secondRound_final['Total_Population']))
df_secondRound_final['NoChildren_Adults_prop'] = list(map(getRatio,df_secondRound_final['NoChildren_Adults'],df_secondRound_final['Total_Population']))
df_secondRound_final['NoChildren_Alone'] = list(map(getRatio,df_secondRound_final['NoChildren_Alone'],df_secondRound_final['Total_Population']))


#Get Voter Ratios:
df_secondRound_final['% Abs/Reg'] = list(map(getRatio,df_secondRound_final['Abstentions'],df_secondRound_final['Registered']))
df_secondRound_final['% Voter/Reg'] = list(map(getRatio,df_secondRound_final['Voters'],df_secondRound_final['Registered']))
df_secondRound_final['% NOTA/Reg'] = list(map(getRatio,df_secondRound_final['None of the above(NOTA)'],df_secondRound_final['Registered']))
df_secondRound_final['% NOTA/Voter'] = list(map(getRatio,df_secondRound_final['None of the above(NOTA)'],df_secondRound_final['Voters']))
df_secondRound_final['% Nulls/Reg'] = list(map(getRatio,df_secondRound_final['Nulls'],df_secondRound_final['Registered']))
df_secondRound_final['% Nulls/Voter'] = list(map(getRatio,df_secondRound_final['Nulls'],df_secondRound_final['Voters']))
df_secondRound_final['% Exp/Reg'] = list(map(getRatio,df_secondRound_final['Expressed'],df_secondRound_final['Registered']))
df_secondRound_final['% Exp/Voter'] = list(map(getRatio,df_secondRound_final['Expressed'],df_secondRound_final['Voters']))


In [71]:
check = df_firstRound_final.iloc[0:100]
check.to_csv('checker.csv')

In [72]:
x = df_firstRound_final.columns.tolist()
x

['Winner',
 'Joiner',
 'EU_circo',
 'code_région',
 'nom_région',
 'chef.lieu_région',
 'nom_département',
 'préfecture',
 'Constituency code',
 'Commune_x',
 'codes_postaux',
 'INSEE code_x',
 'latitude',
 'longitude',
 'éloignement',
 'Department code',
 'Registered',
 'Abstentions',
 'Voters',
 'None of the above(NOTA)',
 'Nulls',
 'Expressed',
 'Commune_x',
 'INSEE code_x',
 'ARTHAUD',
 'ASSELINEAU',
 'CHEMINADE',
 'DUPONT-AIGNAN',
 'FILLON',
 'HAMON',
 'LASSALLE',
 'LE PEN',
 'MACRON',
 'MÉLENCHON',
 'POUTOU',
 'Total_Population',
 'Children_2Parents',
 'Children_1Parent',
 'NoChildren_Couple',
 'Children_Couple',
 'Children_Single',
 'NoChildren_Adults',
 'NoChildren_Alone',
 'Male_pop',
 'Female_pop',
 'Under_20_pop',
 '20-29_pop',
 '30-39_pop',
 '40-49_pop',
 '50-59_pop',
 '60-69_pop',
 '70-79_pop',
 '80Plus_pop',
 'E14TST',
 'E14TS0ND',
 'E14TS1',
 'E14TS6',
 'E14TS10',
 'E14TS20',
 'E14TS50',
 'E14TS100',
 'E14TS200',
 'E14TS500',
 'genderRatio',
 'Age_Under20_prop',
 'Age_20

In [73]:
#Final check
df_firstRound_final = df_firstRound_final.drop_duplicates(subset=['Joiner'])
df_firstRound_final = pd.merge(df_firstRound_final, df_voteDiff, how='left', on='Joiner')
df_firstRound_final = df_firstRound_final[df_firstRound_final['FirstSecond_Diff'] > 0]

df_secondRound_final = df_secondRound_final.drop_duplicates(subset=['Joiner'])

In [74]:
df_firstRound_final = df_firstRound_final.drop(['codes_postaux','éloignement',
                                                'Under_20_pop','20-29_pop','30-39_pop',
                                                '40-49_pop','50-59_pop','60-69_pop',
                                                '70-79_pop','80Plus_pop','Children_2Parents',
                                                'Children_1Parent','NoChildren_Couple','Children_Couple',
                                                'Children_Single','NoChildren_Adults',
                                                'NoChildren_Alone','Male_pop','Female_pop','FirstSecond_Diff',
                                                'FirstPlace_Votes', 'SecondPlace_Votes'], axis=1)

df_secondRound_final = df_secondRound_final.drop(['codes_postaux','éloignement',
                                                'Under_20_pop','20-29_pop','30-39_pop',
                                                '40-49_pop','50-59_pop','60-69_pop',
                                                '70-79_pop','80Plus_pop','Children_2Parents',
                                                'Children_1Parent','NoChildren_Couple','Children_Couple',
                                                'Children_Single','NoChildren_Adults',
                                                'NoChildren_Alone','Male_pop','Female_pop'], axis=1)

In [75]:
#Drop any remaining NA or inf values
df_firstRound_final = df_firstRound_final.replace([np.inf, -np.inf], np.nan)
df_firstRound_final = df_firstRound_final.dropna(how='any')

df_secondRound_final = df_secondRound_final.replace([np.inf, -np.inf], np.nan)
df_secondRound_final = df_secondRound_final.dropna(how='any')

In [76]:
df_firstRound_final_ModelInputs = df_firstRound_final.drop(['EU_circo','code_région','nom_région','chef.lieu_région','nom_département','préfecture','Constituency code','Commune_x',
                                                            'INSEE code_x','latitude','longitude','Commune_x','INSEE code_x','LE PEN','MACRON','ARTHAUD','ASSELINEAU','CHEMINADE',
                                                            'DUPONT-AIGNAN','FILLON','HAMON','LASSALLE','MÉLENCHON','POUTOU'], axis=1)


df_secondRound_final_ModelInputs = df_secondRound_final.drop(['EU_circo','code_région','nom_région','chef.lieu_région','nom_département','préfecture','Constituency code','Commune_x',
                                                            'INSEE code_x','latitude','longitude','Commune_x','INSEE code_x','LE PEN','MACRON'], axis=1)

,Winner,Joiner,Department code,Registered,Abstentions,Voters,None of the above(NOTA),Nulls,Expressed,ARTHAUD,...,NoChildren_Adults_prop,NoChildren_Alone_prop,% Abs/Reg,% Voter/Reg,% NOTA/Reg,% NOTA/Voter,% Nulls/Reg,% Nulls/Voter,% Exp/Reg,% Exp/Voter
0,LE PEN,zuytpeene-59669,59,423.0,47.0,376.0,7.0,2.0,367.0,4,...,0.014862,0.044586,0.111111,0.888889,0.016548,0.018617,0.004728,0.005319,0.867612,0.976064
1,LE PEN,zuydcoote-59668,59,1175.0,148.0,1027.0,23.0,7.0,997.0,16,...,0.041451,0.095855,0.125957,0.874043,0.019574,0.022395,0.005957,0.006816,0.848511,0.970789
2,LE PEN,zutkerque-62906,62,1362.0,285.0,1077.0,23.0,9.0,1045.0,9,...,0.017231,0.072000,0.209251,0.790749,0.016887,0.021356,0.006608,0.008357,0.767254,0.970288
3,FILLON,zudausques-62905,62,677.0,76.0,601.0,10.0,2.0,589.0,2,...,0.014286,0.052381,0.112260,0.887740,0.014771,0.016639,0.002954,0.003328,0.870015,0.980033
4,MACRON,zoufftgen-57764,57,754.0,127.0,627.0,16.0,2.0,609.0,4,...,0.003850,0.116458,0.168435,0.831565,0.021220,0.025518,0.002653,0.003190,0.807692,0.971292
5,LE PEN,zouafques-62904,62,411.0,55.0,356.0,6.0,1.0,349.0,1,...,0.057325,0.076433,0.133820,0.866180,0.014599,0.016854,0.002433,0.002809,0.849148,0.980337
6,LE PEN,zoteux-62903,62,421.0,58.0,363.0,2.0,3.0,358.0,8,...,0.024668,0.056926,0.137767,0.862233,0.004751,0.005510,0.007126,0.008264,0.850356,0.986226
7,MACRON,zommange-57763,57,33.0,2.0,31.0,2.0,0.0,29.0,1,...,0.000000,0.125000,0.060606,0.939394,0.060606,0.064516,0.000000,0.000000,0.878788,0.935484
8,MACRON,zoebersdorf-67560,67,163.0,29.0,134.0,5.0,1.0,128.0,3,...,0.025000,0.075000,0.177914,0.822086,0.030675,0.037313,0.006135,0.007463,0.785276,0.955224
9,LE PEN,zittersheim-67559,67,226.0,47.0,179.0,2.0,3.0,174.0,4,...,0.014778,0.093596,0.207965,0.792035,0.008850,0.011173,0.013274,0.016760,0.769912,0.972067


# WE DONE!!!

# Lets Do some modelling

In [79]:
import sklearn
from sklearn.learning_curve import learning_curve
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn import naive_bayes
from sklearn.metrics import accuracy_score, classification_report


import warnings
warnings.filterwarnings('ignore')

In [83]:
tryThis = df_firstRound_final_ModelInputs.drop(['Joiner'], axis = 1)
tryThis.columns.tolist()

['Winner',
 'Department code',
 'Registered',
 'Abstentions',
 'Voters',
 'None of the above(NOTA)',
 'Nulls',
 'Expressed',
 'ARTHAUD',
 'ASSELINEAU',
 'CHEMINADE',
 'DUPONT-AIGNAN',
 'FILLON',
 'HAMON',
 'LASSALLE',
 'MÉLENCHON',
 'POUTOU',
 'Total_Population',
 'E14TST',
 'E14TS0ND',
 'E14TS1',
 'E14TS6',
 'E14TS10',
 'E14TS20',
 'E14TS50',
 'E14TS100',
 'E14TS200',
 'E14TS500',
 'genderRatio',
 'Age_Under20_prop',
 'Age_20-29_prop',
 'Age_30-39_prop',
 'Age_40-49_prop',
 'Age_50-59_prop',
 'Age_60-69_prop',
 'Age_70-79_prop',
 'Age_80Plus_prop',
 'Children_2Parents_prop',
 'Children_1Parent_prop',
 'NoChildren_Couple_prop',
 'Children_Couple_prop',
 'Children_Single_prop',
 'NoChildren_Adults_prop',
 'NoChildren_Alone_prop',
 '% Abs/Reg',
 '% Voter/Reg',
 '% NOTA/Reg',
 '% NOTA/Voter',
 '% Nulls/Reg',
 '% Nulls/Voter',
 '% Exp/Reg',
 '% Exp/Voter']

In [81]:
cols = list(tryThis.columns.values)
X = tryThis[cols[1:]]
y = tryThis[cols[0]]

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=4444)

In [82]:
#Balanced Random Forest

model = RandomForestClassifier(class_weight='balanced')
model.fit(X_train, y_train)

print("Accuracy: %.3f"% accuracy_score(y_test, model.predict(X_test)))
print(classification_report(y_test, model.predict(X_test)))

Accuracy: 0.666
               precision    recall  f1-score   support

DUPONT-AIGNAN       0.00      0.00      0.00         4
       FILLON       0.69      0.45      0.55      1485
        HAMON       0.00      0.00      0.00         1
     LASSALLE       0.55      0.30      0.39        20
       LE PEN       0.68      0.89      0.77      4898
       MACRON       0.59      0.36      0.45      1828
    MÉLENCHON       0.67      0.42      0.51       838

  avg / total       0.66      0.67      0.64      9074



In [61]:
#Default Random Forest

model = RandomForestClassifier()
model.fit(X_train, y_train)

print("Accuracy: %.3f"% accuracy_score(y_test, model.predict(X_test)))
print(classification_report(y_test, model.predict(X_test)))

Accuracy: 0.664
               precision    recall  f1-score   support

DUPONT-AIGNAN       0.00      0.00      0.00         5
       FILLON       0.60      0.49      0.54      1526
        HAMON       0.00      0.00      0.00         2
     LASSALLE       0.50      0.10      0.16        21
       LE PEN       0.69      0.88      0.77      5317
       MACRON       0.58      0.38      0.46      1992
    MÉLENCHON       0.69      0.37      0.48       917

  avg / total       0.65      0.66      0.64      9780



In [62]:
# Lets try Naive Bayes
model = naive_bayes.GaussianNB()
model.fit(X_train, y_train)

print("Accuracy: %.3f"% accuracy_score(y_test, model.predict(X_test)))
print(classification_report(y_test, model.predict(X_test)))

Accuracy: 0.256
               precision    recall  f1-score   support

DUPONT-AIGNAN       0.01      0.40      0.01         5
       FILLON       0.24      0.02      0.03      1526
        HAMON       0.00      0.50      0.00         2
     LASSALLE       0.00      0.76      0.01        21
       LE PEN       0.57      0.43      0.49      5317
       MACRON       0.48      0.08      0.14      1992
    MÉLENCHON       0.17      0.02      0.04       917

  avg / total       0.46      0.26      0.30      9780



In [63]:
#Lets try Support Vector machine
model = SVC()
model.fit(X_train, y_train)

print("Accuracy: %.3f"% accuracy_score(y_test, model.predict(X_test)))
print(classification_report(y_test, model.predict(X_test)))

Accuracy: 0.546
               precision    recall  f1-score   support

DUPONT-AIGNAN       0.00      0.00      0.00         5
       FILLON       0.83      0.01      0.03      1526
        HAMON       0.00      0.00      0.00         2
     LASSALLE       0.00      0.00      0.00        21
       LE PEN       0.54      1.00      0.71      5317
       MACRON       1.00      0.00      0.00      1992
    MÉLENCHON       0.80      0.00      0.01       917

  avg / total       0.70      0.55      0.39      9780



In [64]:
#Lets try DecisionTree Classifier

model = DecisionTreeClassifier()
model.fit(X_train, y_train)

print("Accuracy: %.3f"% accuracy_score(y_test, model.predict(X_test)))
print(classification_report(y_test, model.predict(X_test)))

Accuracy: 0.639
               precision    recall  f1-score   support

DUPONT-AIGNAN       0.20      0.20      0.20         5
       FILLON       0.63      0.60      0.61      1526
        HAMON       0.00      0.00      0.00         2
     LASSALLE       0.35      0.29      0.32        21
       LE PEN       0.74      0.74      0.74      5317
       MACRON       0.43      0.44      0.44      1992
    MÉLENCHON       0.55      0.57      0.56       917

  avg / total       0.64      0.64      0.64      9780



In [59]:
df_firstRound_final_ModelInputs.groupby('Winner').count()

,Department code,Registered,Abstentions,Voters,None of the above(NOTA),Nulls,Expressed,Total_Population,E14TST,E14TS0ND,...,NoChildren_Adults_prop,NoChildren_Alone_prop,% Abs/Reg,% Voter/Reg,% NOTA/Reg,% NOTA/Voter,% Nulls/Reg,% Nulls/Voter,% Exp/Reg,% Exp/Voter
Winner,,,,,,,,,,,,,,,,,,,,,
ARTHAUD,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
ASSELINEAU,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
DUPONT-AIGNAN,16,16,16,16,16,16,16,16,16,16,...,16,16,16,16,16,16,16,16,16,16
FILLON,5542,5542,5542,5542,5542,5542,5542,5542,5542,5542,...,5542,5542,5542,5542,5542,5542,5542,5542,5542,5542
HAMON,18,18,18,18,18,18,18,18,18,18,...,18,18,18,18,18,18,18,18,18,18
LASSALLE,75,75,75,75,75,75,75,75,75,75,...,75,75,75,75,75,75,75,75,75,75
LE PEN,17820,17820,17820,17820,17820,17820,17820,17820,17820,17820,...,17820,17820,17820,17820,17820,17820,17820,17820,17820,17820
MACRON,6708,6708,6708,6708,6708,6708,6708,6708,6708,6708,...,6708,6708,6708,6708,6708,6708,6708,6708,6708,6708
MÉLENCHON,3245,3245,3245,3245,3245,3245,3245,3245,3245,3245,...,3245,3245,3245,3245,3245,3245,3245,3245,3245,3245


In [60]:
x = df_secondRound.groupby('Surname').count()

In [61]:
x

,Unnamed: 0,Department code,Department,Constituency code,Constituency,Commune code,Commune,Polling station,Registered,Abstentions,...,Polling station name,Address,Postal code,City,Poll.St.-unique,Z,AB,INSEE code,INSEE_noZAB,Joiner
Surname,,,,,,,,,,,,,,,,,,,,,
LE PEN,66019,66019,66019,66019,66019,66019,66019,66019,66019,66019,...,60902,45833,60890,60890,60890,66019,66019,66019,66019,66019
MACRON,66019,66019,66019,66019,66019,66019,66019,66019,66019,66019,...,60902,45833,60890,60890,60890,66019,66019,66019,66019,66019


In [62]:
df_firstRound_final_ModelInputs.dtypes

Winner                      object
Department code             object
Registered                 float64
Abstentions                float64
Voters                     float64
None of the above(NOTA)    float64
Nulls                      float64
Expressed                  float64
Total_Population           float64
E14TST                     float64
E14TS0ND                   float64
E14TS1                     float64
E14TS6                     float64
E14TS10                    float64
E14TS20                    float64
E14TS50                    float64
E14TS100                   float64
E14TS200                   float64
E14TS500                   float64
genderRatio                float64
Age_Under20_prop           float64
Age_20-29_prop             float64
Age_30-39_prop             float64
Age_40-49_prop             float64
Age_50-59_prop             float64
Age_60-69_prop             float64
Age_70-79_prop             float64
Age_80Plus_prop            float64
Children_2Parents_pr

In [63]:
cols = list(df_secondRound_final_ModelInputs.columns.values)
X2 = df_secondRound_final_ModelInputs[cols[1:]]
y2 = df_secondRound_final_ModelInputs[cols[0]]

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2,test_size=0.3, random_state=4444)

In [64]:
#Balanced Random Forest

model = RandomForestClassifier(class_weight='balanced')
model.fit(X_train2, y_train2)

print("Accuracy: %.3f"% accuracy_score(y_test2, model.predict(X_test2)))
print(classification_report(y_test2, model.predict(X_test2)))

Accuracy: 0.735
             precision    recall  f1-score   support

     LE PEN       0.51      0.37      0.43      2702
     MACRON       0.79      0.87      0.83      7331

avg / total       0.71      0.74      0.72     10033



In [65]:
#Default Random Forest

model = RandomForestClassifier()
model.fit(X_train2, y_train2)

print("Accuracy: %.3f"% accuracy_score(y_test2, model.predict(X_test2)))
print(classification_report(y_test2, model.predict(X_test2)))

Accuracy: 0.732
             precision    recall  f1-score   support

     LE PEN       0.50      0.38      0.44      2702
     MACRON       0.79      0.86      0.82      7331

avg / total       0.71      0.73      0.72     10033



In [66]:
# Lets try Naive Bayes
model = naive_bayes.GaussianNB()
model.fit(X_train2, y_train2)

print("Accuracy: %.3f"% accuracy_score(y_test2, model.predict(X_test2)))
print(classification_report(y_test2, model.predict(X_test2)))

Accuracy: 0.324
             precision    recall  f1-score   support

     LE PEN       0.28      0.97      0.44      2702
     MACRON       0.89      0.08      0.15      7331

avg / total       0.73      0.32      0.23     10033



In [95]:
#Lets try Support Vector machine
model = SVC()
model.fit(X_train2, y_train2)

print("Accuracy: %.3f"% accuracy_score(y_test2, model.predict(X_test2)))
print(classification_report(y_test2, model.predict(X_test2)))

Accuracy: 0.726
             precision    recall  f1-score   support

     LE PEN       0.40      0.04      0.07      2701
     MACRON       0.73      0.98      0.84      7348

avg / total       0.65      0.73      0.63     10049



In [67]:
#Lets try DecisionTree Classifier

model = DecisionTreeClassifier()
model.fit(X_train2, y_train2)

print("Accuracy: %.3f"% accuracy_score(y_test2, model.predict(X_test2)))
print(classification_report(y_test2, model.predict(X_test2)))

Accuracy: 0.693
             precision    recall  f1-score   support

     LE PEN       0.43      0.44      0.44      2702
     MACRON       0.79      0.79      0.79      7331

avg / total       0.70      0.69      0.69     10033



In [77]:
df_firstRound_final_ModelInputs.to_csv('df_firstRound_final_ModelInputs.csv')
df_secondRound_final_ModelInputs.to_csv('df_secondRound_final_ModelInputs.csv')

In [78]:
df_firstRound_final.to_csv('df_fistRound_finalData.csv')
df_secondRound_final.to_csv('df_secondRound_finalData.csv')